In [1]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import pickle

import os
path_dir = os.path.dirname(os.getcwd())

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import torch # For building the networks 
import torchtuples as tt # Some useful functions

from pycox.datasets import metabric
from pycox.models import LogisticHazard, PMF, DeepHitSingle, CoxPH
from pycox.evaluation import EvalSurv

%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/Users/linafaik/Documents/survival_analysis/src


In [3]:
from train import *
from train_survival_ml import *
from train_survival_deep import *

In [4]:
# We also set some seeds to make this reproducable.
# Note that on gpu, there is still some randomness.
np.random.seed(1234)
_ = torch.manual_seed(123)

In [5]:
# Parameters

scaler_name = "StandardScaler" #MinMaxScaler
random_state = 123
test_size = 0.3

In [67]:
df = pd.read_csv(os.path.join(path_dir, "outputs/hdhi_clean.csv"))

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning:

Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.



# 1. Train / test split

In [68]:
# covariate columns (used when possible)

cols_x = [
    'age', 'gender', 'rural',
    'duration_of_stay', 'duration_of_intensive_unit_stay', 
    'smoking','alcohol', 'dm', 'htn', 'cad', 'prior_cmp', 'ckd', 'hb', 'tlc',
    'platelets', 'glucose', 'urea', 'creatinine', 'raised_cardiac_enzymes',
    'severe_anaemia', 'anaemia', 'stable_angina', 'acs', 'stemi',
    'atypical_chest_pain', 'heart_failure', 'hfref', 'hfnef', 'valvular',
    'chb', 'sss', 'aki', 'cva_infract', 'cva_bleed', 'af', 'vt', 'psvt',
    'congenital', 'uti', 'neuro_cardiogenic_syncope', 'orthostatic',
    'infective_endocarditis', 'dvt', 'cardiogenic_shock', 'shock',
    'pulmonary_embolism', 'chest_infection',
    'type_adm', 
    'first_visit', 'nb_visits',
    'duration_of_stay_lag1', 'duration_of_intensive_unit_stay_lag1',
    'cardiogenic_shock_lag1', 'cad_lag1', 'time_before_readm_lag1'
]

col_target = "time_before_readm"

In [69]:
Xy_train, Xy_test, y_train, y_test = split_train_test(
    df, cols_x+['doa'], col_target, test_size=0.15, random_state=random_state)

Xy_train, Xy_val, y_train, y_val = split_train_test(
    Xy_train, cols_x, col_target, test_size=0.2, random_state=random_state)

n_train, n_test, n_val = Xy_train.shape[0], Xy_test.shape[0], Xy_val.shape[0]
n_tot =  n_train + n_test + n_val

print("Train: {}%, Test: {}%, Val: {}%".format(
    round(n_train/n_tot *100),
    round(n_test/n_tot *100),
    round(n_val/n_tot *100)
))

Train: 68%, Test: 15%, Val: 17%


In [70]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# rescale
scaler = eval(scaler_name)()

Xy_train[cols_x] = scaler.fit_transform(Xy_train[cols_x])
Xy_test[cols_x] = scaler.transform(Xy_test[cols_x])

In [71]:
#with open(os.path.join(path_dir, "outputs/cox_ph.pkl"), "rb") as f:
    #estimator = pickle.load(f)

In [72]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

# train an estimator
estimator = CoxPHSurvivalAnalysis(alpha=0.5)
estimator = estimator.fit(Xy_train[cols_x], y_train)

In [73]:
estimator.score(Xy_test[cols_x], y_test)

0.6339186792998365

In [74]:
feat_importance, fig = plot_feat_imp(cols_x, estimator.coef_)
fig

# 2. DeepSurv

Source: https://nbviewer.org/github/havakv/pycox/blob/master/examples/cox-ph.ipynb

In [75]:
n_top = 40
cols_x_reduc = list(feat_importance.feature.iloc[:n_top])
cols_x_reduc

['dvt',
 'type_adm',
 'gender',
 'rural',
 'atypical_chest_pain',
 'chb',
 'sss',
 'age',
 'cva_bleed',
 'severe_anaemia',
 'pulmonary_embolism',
 'chest_infection',
 'aki',
 'psvt',
 'congenital',
 'neuro_cardiogenic_syncope',
 'acs',
 'heart_failure',
 'valvular',
 'orthostatic',
 'vt',
 'duration_of_stay_lag1',
 'hfref',
 'stable_angina',
 'htn',
 'cad_lag1',
 'shock',
 'anaemia',
 'infective_endocarditis',
 'first_visit',
 'alcohol',
 'duration_of_intensive_unit_stay_lag1',
 'cva_infract',
 'smoking',
 'tlc',
 'platelets',
 'stemi',
 'prior_cmp',
 'cardiogenic_shock_lag1',
 'raised_cardiac_enzymes']

In [76]:
get_target = lambda df: (df[col_target].values, df['censored'].values)

y_train = get_target(Xy_train)
y_val = get_target(Xy_val)
y_test = get_target(Xy_test)

train = (np.array(Xy_train[cols_x_reduc]).astype(np.float32), y_train)
val = (np.array(Xy_val[cols_x_reduc]).astype(np.float32), y_val)
test = (np.array(Xy_test[cols_x_reduc]).astype(np.float32), y_test)

In [77]:
params = {
    'n_nodes': 264,
    'n_layers': 4,
    'dropout': 0.4,
    'lr':0.005
}


logs_df, model, score = train_deep_surv(
    train, val, test, CoxPH, out_features = 1, tolerance=10, 
    print_lr=True, print_logs=True, verbose = True,
    **params
)

print('score', score)

concordance: 0.49542856307029476
0:	[5s / 5s],		train_loss: 1.9072,	val_loss: 2.4611
1:	[3s / 8s],		train_loss: 1.8467,	val_loss: 3.9352
2:	[2s / 11s],		train_loss: 1.8224,	val_loss: 2.4810
3:	[2s / 14s],		train_loss: 1.8034,	val_loss: 4.5627
4:	[2s / 17s],		train_loss: 1.7895,	val_loss: 3.2483
concordance: 0.5466598272906236
5:	[2s / 20s],		train_loss: 1.7875,	val_loss: 1.8318
6:	[2s / 22s],		train_loss: 1.7824,	val_loss: 2.4341
7:	[2s / 25s],		train_loss: 1.7837,	val_loss: 1.8555
8:	[2s / 27s],		train_loss: 1.7812,	val_loss: 1.8770
9:	[2s / 30s],		train_loss: 1.7823,	val_loss: 2.6103
concordance: 0.4911468917406464
10:	[3s / 33s],		train_loss: 1.7771,	val_loss: 4.4214
11:	[2s / 36s],		train_loss: 1.7844,	val_loss: 1.9348
12:	[3s / 39s],		train_loss: 1.7786,	val_loss: 1.9218
13:	[3s / 42s],		train_loss: 1.7778,	val_loss: 1.8841
14:	[2s / 45s],		train_loss: 1.7759,	val_loss: 4.0899
concordance: 0.5081631181098708
15:	[2s / 48s],		train_loss: 1.7763,	val_loss: 2.0168
16:	[2s / 50s],		tr

score 0.6078104479324722


In [78]:
surv = model.predict_surv_df(test[0])
ev = EvalSurv(surv, test[1][0], test[1][1])
score = ev.concordance_td()
score

0.6078104479324722

In [51]:
grid_params = {
    "n_nodes" :[232, 248, 264],
    "n_layers" :[4, 5],
    "dropout" :[0.3, 0.4],
    "lr" :[0.01, 0.005, 0.001]
}

best_model, table = grid_search_deep(train, val, test, 1, grid_params, CoxPH)

36 total scenario to run
1/36: params: {'n_nodes': 232, 'n_layers': 4, 'dropout': 0.3, 'lr': 0.01}
concordance: 0.5048211044180049
0:	[1s / 1s],		train_loss: 1.9120,	val_loss: 13.9515
1:	[1s / 3s],		train_loss: 1.8116,	val_loss: 2.4332
2:	[1s / 5s],		train_loss: 1.7955,	val_loss: 2.0563
3:	[1s / 7s],		train_loss: 1.7892,	val_loss: 5.8767
4:	[1s / 9s],		train_loss: 1.7911,	val_loss: 5.4681
concordance: 0.4930594960146512
5:	[1s / 10s],		train_loss: 1.7903,	val_loss: 3.2110
6:	[1s / 12s],		train_loss: 1.7906,	val_loss: 1.9036
7:	[1s / 14s],		train_loss: 1.7905,	val_loss: 2.4391
8:	[1s / 16s],		train_loss: 1.7866,	val_loss: 3.9835
9:	[1s / 18s],		train_loss: 1.7884,	val_loss: 2.4450
concordance: 0.49485205814276645
10:	[1s / 20s],		train_loss: 1.7899,	val_loss: 4.1043
11:	[1s / 22s],		train_loss: 1.7880,	val_loss: 2.3542
12:	[1s / 24s],		train_loss: 1.7883,	val_loss: 1.9747
13:	[1s / 25s],		train_loss: 1.7827,	val_loss: 4.2597
14:	[1s / 27s],		train_loss: 1.7879,	val_loss: 4.5053


/usr/local/lib/python3.9/site-packages/pycox/models/cox.py:251: RuntimeWarning:

overflow encountered in exp



concordance: 0.5093204151844236
15:	[2s / 29s],		train_loss: 1.7810,	val_loss: 9690.1289
16:	[2s / 31s],		train_loss: 1.7838,	val_loss: 3.5557
17:	[1s / 33s],		train_loss: 1.7809,	val_loss: 2.5852
18:	[1s / 35s],		train_loss: 1.7841,	val_loss: 1.9255
19:	[1s / 37s],		train_loss: 1.7797,	val_loss: 2.2638
concordance: 0.4995942272841675
20:	[2s / 39s],		train_loss: 1.7796,	val_loss: 60.4871
21:	[2s / 41s],		train_loss: 1.7798,	val_loss: 4.0647
Current score: 0.593094852538029 vs. best score: -100
2/36: params: {'n_nodes': 232, 'n_layers': 4, 'dropout': 0.3, 'lr': 0.005}
concordance: 0.5115538043524861
0:	[2s / 2s],		train_loss: 1.8780,	val_loss: 7.6443
1:	[1s / 4s],		train_loss: 1.8288,	val_loss: 5.6912
2:	[1s / 6s],		train_loss: 1.8067,	val_loss: 1.9125
3:	[1s / 7s],		train_loss: 1.7920,	val_loss: 10.3598
4:	[1s / 9s],		train_loss: 1.7826,	val_loss: 2.0246
concordance: 0.503790658602659
5:	[1s / 11s],		train_loss: 1.7813,	val_loss: 2.2488
6:	[1s / 13s],		train_loss: 1.7779,	val_loss: 2.

/usr/local/lib/python3.9/site-packages/pycox/models/cox.py:251: RuntimeWarning:

overflow encountered in exp



concordance: 0.501403938290267
30:	[2s / 1m:7s],		train_loss: 1.7531,	val_loss: 15.1942
31:	[1s / 1m:9s],		train_loss: 1.7516,	val_loss: 29.0351
32:	[2s / 1m:11s],		train_loss: 1.7504,	val_loss: 275.2091
33:	[1s / 1m:13s],		train_loss: 1.7502,	val_loss: 205.6651
34:	[1s / 1m:15s],		train_loss: 1.7507,	val_loss: 26.4903
concordance: 0.4993334634630545
35:	[1s / 1m:16s],		train_loss: 1.7504,	val_loss: 539.6981
36:	[1s / 1m:18s],		train_loss: 1.7503,	val_loss: 3.3641
Current score: 0.6099321662182666 vs. best score: 0.593094852538029
3/36: params: {'n_nodes': 232, 'n_layers': 4, 'dropout': 0.3, 'lr': 0.001}
concordance: 0.5093738793334333
0:	[2s / 2s],		train_loss: 1.8662,	val_loss: 4.0174
1:	[2s / 4s],		train_loss: 1.8068,	val_loss: 5.7973
2:	[1s / 6s],		train_loss: 1.8063,	val_loss: 5.2317
3:	[1s / 8s],		train_loss: 1.7995,	val_loss: 6.6659
4:	[1s / 9s],		train_loss: 1.7968,	val_loss: 2.8747
concordance: 0.49768666679780516
5:	[1s / 11s],		train_loss: 1.7943,	val_loss: 1.8551
6:	[1s / 1

...

concordance: 0.5001954467711437
15:	[2s / 39s],		train_loss: 1.7875,	val_loss: 10973593.0000
16:	[2s / 42s],		train_loss: 1.7894,	val_loss: 563302.3750
17:	[2s / 45s],		train_loss: 1.7852,	val_loss: 640.6959
18:	[2s / 47s],		train_loss: 1.7814,	val_loss: 1328.6193
19:	[2s / 50s],		train_loss: 1.7804,	val_loss: 84.3618
concordance: 0.5013577876333388
20:	[2s / 52s],		train_loss: 1.7865,	val_loss: 923.4852
21:	[2s / 54s],		train_loss: 1.7795,	val_loss: 224336.1094
22:	[2s / 57s],		train_loss: 1.7796,	val_loss: 1585286.6250
23:	[2s / 59s],		train_loss: 1.7785,	val_loss: 3308414.7500
24:	[2s / 1m:1s],		train_loss: 1.7788,	val_loss: 393131.8750
Current score: 0.602295765488926 vs. best score: 0.6153677942545297
32/36: params: {'n_nodes': 264, 'n_layers': 5, 'dropout': 0.3, 'lr': 0.005}
concordance: 0.4889629316914796
0:	[2s / 2s],		train_loss: 1.9032,	val_loss: 12.1491
1:	[2s / 5s],		train_loss: 1.8476,	val_loss: 1.8649
2:	[2s / 7s],		train_loss: 1.8157,	val_loss: 3.2992
3:	[2s / 9s],		trai

/usr/local/lib/python3.9/site-packages/pycox/models/cox.py:251: RuntimeWarning:

overflow encountered in exp



concordance: 0.49496251709213535
15:	[2s / 37s],		train_loss: 1.7980,	val_loss: 3.6090
16:	[2s / 39s],		train_loss: 1.7937,	val_loss: 9.0001
17:	[2s / 41s],		train_loss: 1.7888,	val_loss: 2300.5085
18:	[2s / 44s],		train_loss: 1.7896,	val_loss: 169.1670
19:	[2s / 46s],		train_loss: 1.7958,	val_loss: 3.3558


/usr/local/lib/python3.9/site-packages/pycox/models/cox.py:231: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.9/site-packages/pycox/models/cox.py:251: RuntimeWarning:

overflow encountered in exp



concordance: 0.03136630659066687
20:	[2s / 48s],		train_loss: 1.7940,	val_loss: 662208.3750
21:	[2s / 50s],		train_loss: 1.7922,	val_loss: 638140.8750
22:	[2s / 52s],		train_loss: 1.7888,	val_loss: 8147.6050
23:	[2s / 55s],		train_loss: 1.7915,	val_loss: 5797.0415
Current score: 0.503055707855713 vs. best score: 0.6153677942545297
35/36: params: {'n_nodes': 264, 'n_layers': 5, 'dropout': 0.4, 'lr': 0.005}
concordance: 0.506300447333526
0:	[2s / 2s],		train_loss: 1.9151,	val_loss: 12.8083
1:	[2s / 4s],		train_loss: 1.8574,	val_loss: 4.1540
2:	[2s / 6s],		train_loss: 1.8250,	val_loss: 1.8475
3:	[2s / 8s],		train_loss: 1.8035,	val_loss: 3.5549
4:	[2s / 10s],		train_loss: 1.7969,	val_loss: 2.0482
concordance: 0.4912094347074124
5:	[2s / 12s],		train_loss: 1.7891,	val_loss: 2.9268
6:	[2s / 14s],		train_loss: 1.7842,	val_loss: 2.9816
7:	[2s / 16s],		train_loss: 1.7845,	val_loss: 2.2286
8:	[2s / 18s],		train_loss: 1.7855,	val_loss: 2.1228
9:	[2s / 20s],		train_loss: 1.7827,	val_loss: 2.0713
c

In [52]:
table

,n_nodes,n_layers,dropout,lr,score
10,232,5,0.4,0.005,0.615368
1,232,4,0.3,0.005,0.609932
28,264,4,0.4,0.005,0.609700
22,248,5,0.4,0.005,0.609517
19,248,5,0.3,0.005,0.607932
7,232,5,0.3,0.005,0.607726
4,232,4,0.4,0.005,0.607076
34,264,5,0.4,0.005,0.606806
21,248,5,0.4,0.010,0.606695
23,248,5,0.4,0.001,0.606534


In [53]:
table.sort_values(by="score", ascending=False, inplace=True)
best_params = table.drop('score', axis=1).iloc[0].to_dict()
best_score = table.score.iloc[0]

print('Best score: ', best_score)
print('Best params: ')
print(best_params)

Best score:  0.6153677942545297
Best params: 
{'n_nodes': 232.0, 'n_layers': 5.0, 'dropout': 0.4, 'lr': 0.005}


# 3. DeepHit

In [57]:
from pycox.models import DeepHitSingle

In [58]:
num_durations = int(df[col_target].max())
labtrans = DeepHitSingle.label_transform(num_durations)

In [59]:
get_target = lambda df: (df[col_target].values, df['censored'].values)

y_train = labtrans.fit_transform(*get_target(Xy_train))
y_val = labtrans.transform(*get_target(Xy_val))

train = (np.array(Xy_train[cols_x_reduc]).astype(np.float32), y_train)
val = (np.array(Xy_val[cols_x_reduc]).astype(np.float32), y_val)
test = (np.array(Xy_test[cols_x_reduc]).astype(np.float32), y_test)

In [60]:
params = {
    'n_nodes': 236,
    'n_layers': 4,
    'out_features': 729,
    'dropout': 0.1,
    'model_params': {'alpha': 0.2, 'sigma': 0.1, 'duration_index': labtrans.cuts},
    'discrete': True,
}

In [61]:
logs_df, model, score = train_deep_surv(
    train, val, test, DeepHitSingle, tolerance=10, 
    print_lr=True, print_logs=True, verbose = True,
    **params
)

concordance: 0.5030442491096127
0:	[4s / 4s],		train_loss: 33.3355,	val_loss: 35.8477
1:	[3s / 8s],		train_loss: 26.5236,	val_loss: 110.4716
2:	[3s / 11s],		train_loss: 29.6387,	val_loss: 97.1768
3:	[2s / 14s],		train_loss: 25.5052,	val_loss: 248.8673
4:	[2s / 17s],		train_loss: 11.9761,	val_loss: 312.6776
concordance: 0.5010419184825589
5:	[3s / 20s],		train_loss: 22.5591,	val_loss: 267.9500
6:	[2s / 23s],		train_loss: 7.9705,	val_loss: 313.3258
7:	[3s / 26s],		train_loss: 6.4626,	val_loss: 263.2984
8:	[2s / 29s],		train_loss: 5.5661,	val_loss: 582.5009
9:	[2s / 32s],		train_loss: 2.7997,	val_loss: 753.6334
concordance: 0.500992207186588
10:	[2s / 35s],		train_loss: 2.9495,	val_loss: 24518.8633
11:	[2s / 38s],		train_loss: 3.4734,	val_loss: 9061.5703
12:	[3s / 41s],		train_loss: 3.6451,	val_loss: 56414.7539
13:	[2s / 44s],		train_loss: 7.8709,	val_loss: 106712.0547
14:	[2s / 46s],		train_loss: 4.8069,	val_loss: 43522.3359
concordance: 0.5014287581613583
15:	[3s / 50s],		train_loss: 2.

In [62]:
grid_params = {
    'n_nodes': [32],
    'n_layers': [4],
    'dropout': [0.1],
    'model_params': [
        {'alpha': 0.01, 'sigma': 0.2, 'duration_index': labtrans.cuts},
        {'alpha': 0.05, 'sigma': 0.2, 'duration_index': labtrans.cuts},
        {'alpha': 0.1, 'sigma': 0.2, 'duration_index': labtrans.cuts},
    ],
    'epochs': [512],
    'batch_size': [16],
    'discrete': [True],
    'output_bias': [True],
}

best_model, table = grid_search_deep(train, val, test, labtrans.out_features, grid_params, DeepHitSingle)

3 total scenario to run
1/3: params: {'n_nodes': 32, 'n_layers': 4, 'dropout': 0.1, 'model_params': {'alpha': 0.01, 'sigma': 0.2}, 'epochs': 512, 'batch_size': 16, 'discrete': True, 'output_bias': True}
concordance: 0.5123649909333662
0:	[2s / 2s],		train_loss: 0.4140,	val_loss: 1.5919
1:	[2s / 4s],		train_loss: 0.4049,	val_loss: 0.6698
2:	[2s / 6s],		train_loss: 0.4028,	val_loss: 0.5355
3:	[2s / 8s],		train_loss: 0.4011,	val_loss: 0.4680
4:	[2s / 11s],		train_loss: 0.4001,	val_loss: 0.5460
concordance: 0.4976244561407582
5:	[2s / 13s],		train_loss: 0.3991,	val_loss: 0.9318
6:	[2s / 15s],		train_loss: 0.3983,	val_loss: 1.3449
7:	[2s / 17s],		train_loss: 0.3981,	val_loss: 0.4674
8:	[2s / 20s],		train_loss: 0.3978,	val_loss: 0.5604
9:	[2s / 22s],		train_loss: 0.3958,	val_loss: 0.4443
concordance: 0.5062444453304439
10:	[3s / 25s],		train_loss: 0.3961,	val_loss: 0.4938
11:	[2s / 28s],		train_loss: 0.3976,	val_loss: 0.4298
12:	[2s / 30s],		train_loss: 0.3960,	val_loss: 0.4702
13:	[2s / 33s

In [63]:
table

,n_nodes,n_layers,dropout,model_params,epochs,batch_size,discrete,output_bias,lr,score
2,32,4,0.1,"{'alpha': 0.1, 'sigma': 0.2}",512,16,True,True,0.01,0.620541
0,32,4,0.1,"{'alpha': 0.01, 'sigma': 0.2}",512,16,True,True,0.01,0.616121
1,32,4,0.1,"{'alpha': 0.05, 'sigma': 0.2}",512,16,True,True,0.01,0.608533


# Draft

In [ ]:
# finding the best learning rate from this model
lrfinder = model.lr_finder(train[0], train[1], batch_size, tolerance=10)
lr = lrfinder.get_best_lr()
model.optimizer.set_lr(lr)

lrfinder_df = lrfinder.to_pandas()
fig = px.line(x=lrfinder_df.index, y=lrfinder_df.train_loss, 
              log_x=True, width=700, height=400)

fig.update_layout(dict(xaxis={'title':'lr'}, yaxis={'title':'batch_loss'}))
fig.show()

print("Best learning rate: ", lr)

In [ ]:
callbacks = [tt.callbacks.EarlyStopping(patience=15)]

log = model.fit(train[0], train[1], batch_size, 100, callbacks, verbose,
            val_data=val, val_batch_size=batch_size)

In [ ]:
logs_df = log.to_pandas().reset_index().melt(
    id_vars="index", value_name="loss", var_name="dataset").reset_index()

fig = px.line(logs_df, y="loss", x="index", color="dataset", width=800, height = 400)
fig.show()

# scoring the model
surv = model.interpolate(10).predict_surv_df(test[0])
ev = EvalSurv(surv, test[1][0], test[1][1], censor_surv='km')
score = ev.concordance_td()
score

In [ ]:

best_score

In [ ]:
# finding the best learning rate from this model
lrfinder = model.lr_finder(train[0], y_train, params['batch_size'], tolerance=10, lr_min=0.001, lr_max=1.0)

lrfinder_df = lrfinder.to_pandas()
fig = px.line(x=lrfinder_df.index, y=lrfinder_df.train_loss, 
              log_x=True, width=700, height=400)
fig.update_layout(dict(xaxis={'title':'lr'}, yaxis={'title':'batch_loss'}))


print("Best LR: ", lrfinder.get_best_lr())
fig

In [ ]:
help(model.lr_finder)

In [ ]:
# setting the new number
model.optimizer.set_lr(lrfinder.get_best_lr())
model.optimizer.param_groups[0]['lr']

In [ ]:
callbacks = [tt.callbacks.EarlyStopping()]
verbose = True

log = model.fit(train[0], train[1], params['batch_size'], 2, callbacks, verbose,
                val_data=val, val_batch_size=params['batch_size'])

In [ ]:
logs_df = log.to_pandas().reset_index().melt(
    id_vars="index", value_name="loss", var_name="dataset").reset_index()

px.line(logs_df, y="loss", x="index", color="dataset", width=800, height = 400)

In [ ]:
model.partial_log_likelihood(*val).mean()

In [ ]:
_ = model.compute_baseline_hazards()
surv = model.predict_surv_df(test[0])

In [ ]:
N = 3
surv_df = surv[np.random.choice(surv.columns, N)]\
    .reset_index().melt(id_vars="duration", var_name="patient_id", value_name="S")
px.line(surv_df, x="duration", y="S", color="patient_id", width=800, height = 400)

In [ ]:
ev = EvalSurv(surv, test[1][0], test[1][1], censor_surv='km')

In [ ]:
test[1][0].shape[0]

In [ ]:
test[0][0].shape

In [ ]:
surv.shape[1]

In [ ]:
ev.concordance_td()

In [ ]:
time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
brier_scores = ev.brier_score(time_grid)

In [ ]:
brier_scores_df = pd.DataFrame(brier_scores).reset_index().rename(columns={"index":"duration"})
px.line(brier_scores_df, x="duration", y="brier_score", width=800, height = 400)

In [ ]:
time_grid = np.arange(1, 91)

In [ ]:
ev.integrated_brier_score(time_grid)

In [ ]:
ev.integrated_nbll(time_grid)

In [ ]:
help(ev.integrated_nbll)